# Delta Lake

Delta Lake é uma solução de armazenamento de dados open-source da Databricks que traz transações ACID (Atomicidade, Consistência, Isolamento, Durabilidade) para o Apache Spark e big data workloads. Ele oferece um conjunto de funcionalidades adicionais sobre o Apache Parquet, transformando o armazenamento de dados em uma camada de armazenamento altamente confiável, escalável e seguro.

## Recursos Principais:

- **Atomicidade:** As transações são atômicas, o que significa que são bem-sucedidas como uma unidade ou falham e não deixam dados corrompidos.
- **Consistência:** Delta Lake suporta operações ACID para manter a consistência dos dados.
- **Isolamento:** As transações são isoladas umas das outras.
- **Durabilidade:** Os dados gravados são duráveis e podem ser lidos por outras operações mesmo em caso de falhas.

# Upsert (Inserir ou Atualizar)

**Operação de Upsert (Inserir ou Atualizar):** O upsert é uma operação que permite inserir novos registros em um banco de dados ou atualizar os registros existentes se já estiverem presentes na base de dados. No contexto do Delta Lake, a operação de upsert geralmente envolve a atualização de registros existentes com base nas chaves de identificação e a inserção de novos registros se eles não existirem.

## Comparação de Casos de Uso:

| Cenário               | Uso Delta Lake                             | Uso Padrão (Sem Delta Lake)                 |
|-----------------------|--------------------------------------------|--------------------------------------------|
| **Atomicidade**       | Suporta transações ACID.                   | Transações podem não ser atômicas, o que pode levar a dados inconsistentes em caso de falha. |
| **Consistência**      | Mantém a consistência dos dados.           | Pode resultar em dados inconsistentes se uma operação falhar no meio do processo. |
| **Isolamento**        | Oferece isolamento entre transações.       | Pode não oferecer isolamento, o que pode levar a resultados inesperados. |
| **Durabilidade**      | Os dados gravados são duráveis.           | Pode haver perda de dados em caso de falha. |
| **Operação de Upsert**| Permite inserir ou atualizar registros de forma segura. | Pode ser difícil de implementar de forma segura sem mecanismos específicos como Delta Lake. |

# Vacuum

O `vacuum` é um recurso que permite a você excluir dados antigos de um conjunto de dados Delta. O `vacuum` é executado automaticamente em segundo plano, mas também pode ser executado manualmente.

## Casos de uso:

- Liberação de espaço em disco: O `vacuum` pode ser usado para liberar espaço em disco em um conjunto de dados Delta.
- Melhoria do desempenho: O `vacuum` pode melhorar o desempenho de consultas em um conjunto de dados Delta.

# Leitura/escrita e setup

In [2]:
import delta

In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("Exemplo")
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.3.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

In [4]:
spark.sparkContext.addPyFile("delta-core_2.13-2.3.0.jar")

In [5]:
items = spark.sparkContext.getConf().getAll()

for item in items:
    if "jars" in item[0]:
        print(item)

('spark.repl.local.jars', 'file:///home/jovyan/.ivy2/jars/io.delta_delta-core_2.12-2.3.0.jar,file:///home/jovyan/.ivy2/jars/io.delta_delta-storage-2.3.0.jar,file:///home/jovyan/.ivy2/jars/org.antlr_antlr4-runtime-4.8.jar')
('spark.jars.packages', 'io.delta:delta-core_2.12:2.3.0')
('spark.jars', 'file:///home/jovyan/.ivy2/jars/io.delta_delta-core_2.12-2.3.0.jar,file:///home/jovyan/.ivy2/jars/io.delta_delta-storage-2.3.0.jar,file:///home/jovyan/.ivy2/jars/org.antlr_antlr4-runtime-4.8.jar')


## Primeiramente vamos salvar o dad para que depois possamos ler

In [3]:
df = (
    spark
    .read
    .format("parquet")
    .load("/home/app/data/1.bronze/vra/")
)

In [9]:
(
    df.write
    .format("delta")
    .mode("append")
    .option("mergeSchema", "true")
    .save("/home/app/data/3.gold/vra_test/")
)

## Com dado salvo em delta podemos le-lô em delta com a spark session ou mesmo como engine do delta

In [4]:
df_delta = (
    spark
    .read
    .format("delta")
    .load("/home/app/data/3.gold/vra_test/")
)

In [5]:
delta_table = delta.DeltaTable.forPath(spark, "/home/app/data/3.gold/vra_test/")

# Transações

In [ ]:
 # lendo parquet de transações
df_transacao = (
    spark
    .read
    .format("json")
    .load("data/3.gold/vra_test/_delta_log/*.json")
)

df_transacao.show(70,vertical=True,truncate=False)

### ACID transactions 

In [8]:
df_delta.show(1, vertical=True)

-RECORD 0----------------------------------
 icao_empresa_aerea     | AAL              
 numero_voo             | 904              
 codigo_di              | 0                
 codigo_tipo_linha      | I                
 icao_aerodromo_origem  | SBGL             
 icao_aerodromo_destino | KMIA             
 partida_prevista       | 01/07/2022 23:00 
 partida_real           | 03/07/2022 01:33 
 chegada_prevista       | 02/07/2022 07:45 
 chegada_real           | 03/07/2022 09:45 
 situacao_voo           | REALIZADO        
 codigo_justificativa   | null             
 preco_passagem         | null             
 preco_combustivel      | null             
only showing top 1 row



In [9]:
# Realizando a operaçăo de delete
delta_table.delete( "chegada_prevista < '01/01/2022'")

In [ ]:
# Realizando a operação de Update
(
    delta_table.alias("delta")
    .merge(
        df.alias("new_data"),
        "delta.numero_voo = new_data.numero_voo"
    )
    # .whenMatchedUpdate(
    #     set= {"partida_real": "new_data.partida_real"}
    # )
    .whenMatchedUpdateAll()
    .execute( )
)

In [ ]:
# Realizando a operação de Upsert
# Tentando inserir o registro; se já existir, ele será atualizado
(
    delta_table.alias("delta")
    .merge(
        df.alias("new_data"),
        "delta.numero_voo = new_data.numero_voo"
    )
    # .whenMatchedUpdate(
    #     set= {"partida_real": "new_data.partida_real"}
    # )
    .whenMatchedUpdateAll()
    # .whenNotMatchedInsert(
    #     values={"numero_voo": "new_data.numero_voo", "icao_empresa_aerea": "new_data.icao_empresa_aerea"}
    # )
    .whenNotMatchedInsertAll()
    .execute()
)

# Change data feed

In [6]:
(
    spark
    .readStream
    .format("delta")
    .option("readChangefeed","true")
    .option("staringVersion", 0)
    .load("/home/app/data/3.gold/vra_test/")
)

DataFrame[icao_empresa_aerea: string, numero_voo: string, codigo_di: string, codigo_tipo_linha: string, icao_aerodromo_origem: string, icao_aerodromo_destino: string, partida_prevista: string, partida_real: string, chegada_prevista: string, chegada_real: string, situacao_voo: string, codigo_justificativa: string, preco_passagem: float, preco_combustivel: double, _change_type: string, _commit_version: bigint, _commit_timestamp: timestamp]

# Rollback

In [8]:
delta_table.history().show(truncate=False, vertical=True)

-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 13                                                                                                                                                                                                                                                             
 timestamp           | 2023-11-20 17:53:40.971                                                                                                                                                                                                                                        
 userId              | null                                                                                                                                        

In [9]:
delta_table.restoreToVersion(1)

DataFrame[table_size_after_restore: bigint, num_of_files_after_restore: bigint, num_removed_files: bigint, num_restored_files: bigint, removed_files_size: bigint, restored_files_size: bigint]

# Vacuum

In [10]:
delta_table.vacuum(1)

IllegalArgumentException: requirement failed: Are you sure you would like to vacuum files with such a low retention period? If you have
writers that are currently writing to this table, there is a risk that you may corrupt the
state of your Delta table.

If you are certain that there are no operations being performed on this table, such as
insert/upsert/delete/optimize, then you may turn off this check by setting:
spark.databricks.delta.retentionDurationCheck.enabled = false

If you are not sure, please use a value not less than "168 hours".
       